# ANEXO 05

## Modelo de series temporales

El objetivo de este codigo es modelar los datos procesados en el notebook "data_preparation_anexo.ipynb". Además de los datos procesados, también se recoge el filtro creado en el mismo notebook para poder aplicar el modelo a todos los diferentes escenarios requeridos por naturgy.

In [1]:
# General
import pandas as pd
import numpy as np
#---------------------------------------------------------

# Modelo de series temporales que vamos a utilizar.
from prophet import Prophet
#---------------------------------------------------------

# Eliminamos los warnings del notebook
import warnings
warnings.filterwarnings('ignore')


## Carga del dataset y del filtro generado

In [3]:
%store -r df_temporal_series
df = df_temporal_series.copy() 

%store -r nfilasXcombinacion_selected
nfilasXcombinacion_selected = nfilasXcombinacion_selected

## Ejecucion del modelo para obtener predicciones.

El siguiente trozo de codigo está dividido en dos partes. Esto es debido a las limitaciones del kernel de jupyter notebook para procesar todos los datos a la vez. Al menos en la manera en la que lo hemos intentado nosotros.

El bucle aplicará el modelo sobre cada una de las columnas indicadas. Recogerá las predicciones de cada escenarios posible aplicando el filtro anteriormente mencionado, y las unirá a un dataframe temporal. Se calculan aproximadamente 1000 escenarios diferentes. 

Este dataframe temporal a su vez se añadirá a un dataframe definitivo en el que se irán añadiendo todos los datos reales y predichos de todos los escenarios con el formato necesario para poder representarse en power bi.

Este codigo tarda en ejercutarse aproximadamente 1h y 30 min y devolverá seis archivos ".csv" para cada una de las columnas predichas. 

In [ ]:
df_full_predictions = pd.DataFrame()

columns = ['Altas M', 'Altas_Mes_C+T', 'Altas_Mes_Tra', 'Bajas M', 'Bajas_Mes_C+T', 'Bajas_Mes_Tra']

for column in columns:

    df_predictions = pd.DataFrame()

    for combinacion in nfilasXcombinacion_selected:

        Var_split = str(combinacion).split('_')
            
        ID_SOCIEDAD = Var_split[0]
        ID_SEGMENTO = Var_split[1]
        SEGMENTO2 = Var_split[2]
        ID_LINEA_PRODUCTO_AGREG = float(Var_split[3].split(':')[1])
        ID_AGRUP_PRODUCTO = float(Var_split[4].split(':')[1])
        ID_AGRUP_PROD = Var_split[5]
        ID_AGRUP_CANAL = float(Var_split[6].split(':')[1])
        ID_ZONA = Var_split[7]

        temp_df = df[(df.ID_SOCIEDAD == ID_SOCIEDAD) & 
                                            (df.ID_SEGMENTO == ID_SEGMENTO) & 
                                            (df.SEGMENTO2 == SEGMENTO2) &
                                            (df.ID_LINEA_PRODUCTO_AGREG == ID_LINEA_PRODUCTO_AGREG) & 
                                            (df.ID_AGRUP_PRODUCTO == ID_AGRUP_PRODUCTO) &
                                            (df.ID_AGRUP_PROD == ID_AGRUP_PROD) & 
                                            (df.ID_AGRUP_CANAL == ID_AGRUP_CANAL) & 
                                            (df.ID_ZONA == ID_ZONA)]

        # Seleccionando las columnas necesarias para aplicar prophet
        temp_df_set = temp_df[['fecha_semanas', column]]

        # Renombrando columnas para poder aplicar prophet
        temp_df_set.rename(columns = {'fecha_semanas' : 'ds', column : 'y'}, inplace=True)

        # Creando un modelo temporal prophet
        temp_model = Prophet()

        # Ajustando el modelo de prophet
        temp_model.fit(temp_df_set)

        # Generando el objeto para poder hacer predicciones
        temp_future = temp_model.make_future_dataframe(periods = 133, freq = 'W')

        # Haciendo predicciones
        temp_forecast = temp_model.predict(temp_future)

        # renaming columns to get the final dataframe
        temp_forecast.rename(columns = {'ds' : 'fecha_semanas'}, inplace=True)


        # Dataframe temporal con los datos predichos de un escenario
        temp_df_predictions = pd.merge(temp_df, 
                            temp_forecast[['fecha_semanas', 'yhat', 'yhat_lower', 'yhat_upper']],
                            how= 'right',
                            on = 'fecha_semanas')

        # Este  codigo recoge solo las variables necesarias para BI desde el dataset original. 
        # Genera el dataset final con todos los escenarios posibles para cada columna
        # Y añade datos de los distintos escenarios para las predicciones para poder visualizarlo en BI
        df_predictions = df_predictions.append(temp_df_predictions.sort_values('fecha_semanas')[['ID_SOCIEDAD', 'ID_SEGMENTO', 'SEGMENTO2', 'ID_LINEA_PRODUCTO_AGREG', 'ID_AGRUP_PRODUCTO', 'ID_AGRUP_PROD',
                                                'ID_AGRUP_CANAL', 'ID_ZONA']].fillna(method='ffill').join(temp_df_predictions.sort_values('fecha_semanas')['Cartera_Acu']).join(temp_df_predictions.sort_values('fecha_semanas')[[column,
                                                'fecha_semanas', 'yhat', 'yhat_lower', 'yhat_upper']]))
    

    # Renombramos columnas segun la columna predicha
    df_predictions.rename(columns = {'yhat' : 'yhat_' + column, 'yhat_lower' : 'yhat_lower_' + column, 'yhat_upper': 'yhat_upper_' + column}, inplace=True)
    
    
    # Renombramos las columnas para que sean compatibles con Power BI
    if column == 'Bajas_Mes_C+T':
            df_predictions.rename(columns = {column: 'Bajas_Mes_CT', 'yhat_' + column : 'yhat_Bajas_Mes_CT', 'yhat_lower_' + column  : 'yhat_lower_Bajas_Mes_CT', 'yhat_upper_' + column : 'yhat_upper_Bajas_Mes_CT'}, inplace=True)
    if column == 'Altas_Mes_C+T':
            df_predictions.rename(columns = {column: 'Altas_Mes_CT', 'yhat_' + column  : 'yhat_Altas_Mes_CT', 'yhat_lower_' + column  : 'yhat_lower_Altas_Mes_CT', 'yhat_upper_' + column : 'yhat_upper_Altas_Mes_CT'}, inplace=True)
  

    # Guardamos las predicciones en un archivo .csv
    df_predictions.to_csv('dataframes/full_data_predictions_' + column + '.csv', encoding = 'utf-8', sep=';', decimal=',', index = False)
     

Como se ha comentado en el apartado anterior, debido a las limitaciones del kernel de jupyter se almacenan los archivos ".csv" en local. Una vez almacenados esta parte del codigo consta en cargar dichos archivos y unirlos para crear un dataset final con todas las predicciones.

In [5]:
df_predictions_altas_m = pd.read_csv('../dataframes/full_data_predictions_Altas M.csv', encoding = 'utf-8', sep = ';')
df_predictions_altas_m = pd.read_csv('../dataframes/full_data_predictions_Altas M.csv', encoding = 'utf-8', sep = ';')
df_predictions_bajas_m = pd.read_csv('../dataframes/full_data_predictions_Bajas M.csv', encoding = 'utf-8', sep = ';')
df_predictions_altas_mes_ct = pd.read_csv('../dataframes/full_data_predictions_Altas_Mes_C+T.csv', encoding = 'utf-8', sep = ';')
df_predictions_bajas_mes_ct = pd.read_csv('../dataframes/full_data_predictions_Bajas_Mes_C+T.csv', encoding = 'utf-8', sep = ';')
df_predictions_altas_mes_tra = pd.read_csv('../dataframes/full_data_predictions_Altas_Mes_Tra.csv', encoding = 'utf-8', sep = ';')
df_predictions_bajas_mes_tra = pd.read_csv('../dataframes/full_data_predictions_Bajas_Mes_Tra.csv', encoding = 'utf-8', sep = ';')



# Uniendo todos los dataframes en uno solo
df_full_predictions = pd.merge(df_predictions_altas_m, 
                        df_predictions_bajas_m[['Bajas M', 'yhat_Bajas M', 'yhat_lower_Bajas M', 'yhat_upper_Bajas M']],
                        how= 'left',
                        left_index = True,
                        right_index= True)

df_full_predictions = pd.merge(df_full_predictions, 
                        df_predictions_altas_mes_ct[['Altas_Mes_CT', 'yhat_Altas_Mes_CT', 'yhat_lower_Altas_Mes_CT', 'yhat_upper_Altas_Mes_CT']],
                        how= 'left',
                        left_index = True,
                        right_index= True)

df_full_predictions = pd.merge(df_full_predictions, 
                        df_predictions_bajas_mes_ct[['Bajas_Mes_CT', 'yhat_Bajas_Mes_CT', 'yhat_lower_Bajas_Mes_CT', 'yhat_upper_Bajas_Mes_CT']],
                        how= 'left',
                        left_index = True,
                        right_index= True)

df_full_predictions = pd.merge(df_full_predictions, 
                        df_predictions_altas_mes_tra[['Altas_Mes_Tra', 'yhat_Altas_Mes_Tra', 'yhat_lower_Altas_Mes_Tra', 'yhat_upper_Altas_Mes_Tra']],
                        how= 'left',
                        left_index = True,
                        right_index= True)

df_full_predictions = pd.merge(df_full_predictions, 
                        df_predictions_bajas_mes_tra[['Bajas_Mes_Tra', 'yhat_Bajas_Mes_Tra', 'yhat_lower_Bajas_Mes_Tra', 'yhat_upper_Bajas_Mes_Tra']],
                        how= 'left',
                        left_index = True,
                        right_index= True)




## Correccion de predicciones

Al realizar las predicciones, algunos valores de altas o de bajas quedan como negativos o positivos respectivamente. Dado que esto no tiene sentido para el modelo de negocio el siguiente codigo corrige todos los valores inferiores a 0 en las altas y los superiores a 0 en las bajas.

In [3]:
# Si hay columnas con el valor de altas en negativo, son transformadas a 0 que es el límite que tiene sentido
for column in df_full_predictions.columns:
    if 'Altas' in column:
        temp_np_array = np.array(df_full_predictions[column].values.tolist())
        df_full_predictions[column] = np.where(temp_np_array < 2, 0, temp_np_array).tolist()

# Si hay columnas con el valor de bajas en positivo, son transformadas a 0 que es el límite que tiene sentido
for column in df_full_predictions.columns:
    if 'Bajas' in column:
        temp_np_array = np.array(df_full_predictions[column].values.tolist())
        df_full_predictions[column] = np.where(temp_np_array > -2, 0, temp_np_array).tolist()


In [4]:
# Guardando el dataframe final
df_full_predictions.to_csv('../dataframes/full_data_predictions.csv', encoding = 'utf-8', sep=';', decimal=',', index = False)